In [10]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from tqdm.notebook import trange

import matplotlib.pyplot as plt
import matplotlib as mpl

from function.conserve import Entropy
from function.seqfilter import SeqFilter
from function.utilities import find_human_sequence

# param

In [11]:
#####CHANGE HERE#####
uniprot_id = 'Q13148'
algorithm = 'VLXT'
#####CHANGE HERE#####

In [12]:
tax_ids = [7711,7742,32523,40674]

input_path = Path("./output/extre_filtered_fasta")

disorder_filter_length = 40
order_filter_length = 10

pondr_disorder_ident_path = Path("./output/pondr_disorder_ident")
od_human_df = pd.read_pickle(pondr_disorder_ident_path/'{}_od_human_df.pkl'.format(algorithm))

# conservation level

In [13]:
def get_only_human_score(score,fasta_path):
    '''
    只拿human有的分數，gap都不拿
    
    score: entropy.alied_entropy做的分數
    fasta_path: path
    '''
    nogap_score = []
    nogap = np.array(list(find_human_sequence(fasta_path))) != '-'
    nogap_index = np.where(nogap)[0].tolist()
    for index in nogap_index:
        nogap_score.append(score[index])    
    return nogap_score

In [42]:
def get_score_odregion_human_nogap(taxid,
                                   uniprot_id):

    #read fasta
    fasta_path = input_path / str(taxid) /"{}.fasta".format(uniprot_id)
    
    #get score
    conserve_score = entropy.alied_entropy(fasta_path)
    print(conserve_score)
    
    #only_nogap_index
    conserve_score = get_only_human_score(conserve_score,fasta_path)
    print(conserve_score)
    
    #get order/disorder region
    od_ident = od_human_df[od_human_df['uniprot_id'] == uniprot_id]['od_ident'].values[0] #拿到od序列
    print(len(od_ident))
    od_ident = seqfilter.length_filter_by_od_ident(od_ident,disorder_filter_length,order_filter_length) #篩長度
    
    #score and od_ident length err exception
    #因為oma資料庫的關係，名字一樣，長度會不一樣
    if len(conserve_score) != len(od_ident):
        print("{} ENTROPY LENGTH IS NOT EQUAL WITH OD_IODENT".format(uniprot_id))
        raise Exception("error")
    
    #human_sequence, remove gap
    human_sequence = find_human_sequence(fasta_path)
    human_sequence = human_sequence.replace('-','')
    
    #get region after remove gap
    region = seqfilter.get_od_index(od_ident)
    
    
    #get info
    taxonomy = get_taxid_dict()[taxid]
    protein_name = get_protein_name(uniprot_id)['protein_name']
    gene_name = get_protein_name(uniprot_id)['gene_name']
    
    return {"uniprot_id":uniprot_id,
            "taxonomy":taxonomy,
            "score":conserve_score,
            "od_ident":od_ident,
            "human_sequence":human_sequence,
            "region":region}

In [43]:
entropy = Entropy()
seqfilter = SeqFilter()

In [44]:
get_score_odregion_human_nogap(7711,"Q13148")

[0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.174, 0.753, 0.783, 0.755, 1.0, 0.949, 0.979, 0.979, 0.829, 0.958, 0.865, 1.0, 1.0, 0.824, 0.012, 1.0, 1.0, 0.835, 1.0, 0.979, 1.0, 0.958, 1.0, 0.979, 0.979, 1.0, 1.0, 0.958, 0.958, 1.0, 1.0, 0.979, 0.979, 0.809, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.963, 0.858, 1.0, 0.852, 0.979, 0.963, 0.901, 0.979, 0.963, 1.0, 1.0, 1.0, 0.979, 0.979, 0.979, 0.979, 0.963, 1.0, 0.872, 1.0, 0.979, 0.926, 1.0, 0.788, 0.012, 0.012, 0.757, 0.937, 1.0, 0.979, 0.979, 0.823, 0.921, 0.963, 0.937, 1.0, 0.963, 0.963, 1.0, 1.0, 0.884, 0.063, 0.071, 0.023, 0.023, 0.012, 0.012, 0.012, 0.023, 0.044, 0.054, 0.055, 1.0, 0.979, 1.0, 1.0, 0.979, 0.895, 0.958, 0.756, 0.958, 0.937, 0.907, 0.979, 0.925, 0.942, 0.958, 0.74, 0.958, 0.946, 0.791, 0.809, 0.801, 1.0, 0.942, 0.979, 0.979, 0.979, 0.979, 0.967, 0.967, 0.967, 0.967, 0.979, 0.979, 0.958, 0.958, 0.905, 0.979, 0.958, 0.979, 0.979, 0.958, 0

Exception: error